In [3]:
!git clone https://github.com/fizyr/keras-retinanet.git

fatal: destination path 'keras-retinanet' already exists and is not an empty directory.


In [4]:
!pip install --upgrade keras

Requirement already up-to-date: keras in /usr/local/lib/python3.6/dist-packages (2.3.1)


In [5]:
%cd keras-retinanet
!pip3 install .
!python3 setup.py build_ext --inplace
%cd ..

/root/sketchIT/keras-retinanet
Processing /root/sketchIT/keras-retinanet
  Created wheel for keras-retinanet: filename=keras_retinanet-0.5.1-cp36-cp36m-linux_x86_64.whl size=167666 sha256=5ecacca56a16ed6c03e9a2c312ae991c1b1ac411b104d1e41a594178ddf4e8b3
  Stored in directory: /root/.cache/pip/wheels/96/17/65/c843caa50fcca6a8e23c2793ec6f1411469726a687a24c4f41
Successfully built keras-retinanet
  Attempting uninstall: keras-retinanet
    Found existing installation: keras-retinanet 0.5.1
    Uninstalling keras-retinanet-0.5.1:
      Successfully uninstalled keras-retinanet-0.5.1
running build_ext
cythoning keras_retinanet/utils/compute_overlap.pyx to keras_retinanet/utils/compute_overlap.c
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /root/sketchIT/keras-retinanet/keras_retinanet/utils/compute_overlap.pyx
  tree = Parsing.p_module(s, pxd, full_

In [6]:
!pip install tensorflow-gpu

In [7]:
!pip install pandas
!pip install seaborn
!pip install sklearn
!pip install opencv-python

In [8]:
import numpy as np
import tensorflow as tf
import pandas as pd
import json
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters
from sklearn.model_selection import train_test_split

import urllib
import os
import csv
import cv2
import time
from PIL import Image

from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

%matplotlib inline
%config InlineBackend.figure_format='retina'

register_matplotlib_converters()
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 22, 10

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

In [9]:
os.makedirs("snapshots", exist_ok=True)
os.makedirs("models", exist_ok=True)

In [13]:
TRAINING_DATA_PATH = './model'

with open(os.path.join(TRAINING_DATA_PATH, 'dataset.json')) as f:
   data = json.load(f)

dataset_df = pd.DataFrame(data)

In [14]:
dataset_df.head()

,id,created,width,height,imageUri,thumbnailUri,tags,regions
0,2469da39-65f0-428b-a1de-cedbba7db32e,2018-07-11T07:27:39.1873304Z,1600,564,None,None,[{'tagId': '530d20f2-7260-42f1-bd42-bddca465c9...,[{'regionId': '058d3c7a-a9fd-4c76-9f05-9eaff02...
1,22b40473-2892-4ab9-96be-9aee12ffd17a,2018-07-11T07:27:39.0942122Z,1600,564,None,None,[{'tagId': '530d20f2-7260-42f1-bd42-bddca465c9...,[{'regionId': '054c2b0c-a102-4908-9d09-fcf81b6...
2,6209fb2c-ec39-4f8e-b86b-075cc181a81c,2018-07-11T07:27:38.2810459Z,1600,270,None,None,[{'tagId': '1599c0ca-83c0-4da7-b8c7-2a0a4b069f...,[{'regionId': '1609a4ed-81fa-4ef8-8dd8-7f45d84...
3,13b9f109-f532-471a-8f39-f19eef551919,2018-07-11T07:27:38.2030543Z,1600,270,None,None,[{'tagId': '1599c0ca-83c0-4da7-b8c7-2a0a4b069f...,[{'regionId': '115d74e0-6dd6-4b6d-b48b-9086018...
4,f1f0f46b-4430-4227-ab3a-24ad1192728e,2018-07-11T07:27:38.1873131Z,1600,277,None,None,[{'tagId': '1599c0ca-83c0-4da7-b8c7-2a0a4b069f...,[{'regionId': '0365e098-ca4a-432a-baf9-a4faa0b...


In [15]:
from collections import OrderedDict

dataset = OrderedDict()
dataset["image_name"] = list()
dataset["x_min"] = list()
dataset["y_min"] = list()
dataset["x_max"] = list()
dataset["y_max"] = list()
dataset["class_name"] = list()

for index, row in dataset_df.iterrows():
    image_path = os.path.join(TRAINING_DATA_PATH, 'images') + '/' + row.id + '.png'

    for region in row['regions']:
        x_min = int(region['left'] * row['width'])
        y_min = int(region['top'] * row['height'])

        width = region['width'] * row['width']
        height = region['height'] * row['height']
        x_max = int(x_min + width)
        y_max = int(y_min + height)

        dataset["image_name"].append(image_path)
        dataset['x_min'].append(x_min)
        dataset['y_min'].append(y_min)
        dataset['x_max'].append(x_max)
        dataset['y_max'].append(y_max)
        dataset['class_name'].append(region['tagName'])

In [16]:
df = pd.DataFrame(dataset)
df.head()

,image_name,x_min,y_min,x_max,y_max,class_name
0,./model/images/2469da39-65f0-428b-a1de-cedbba7...,423,332,690,414,CheckBox
1,./model/images/2469da39-65f0-428b-a1de-cedbba7...,989,422,1186,554,Button
2,./model/images/2469da39-65f0-428b-a1de-cedbba7...,18,146,328,207,Label
3,./model/images/2469da39-65f0-428b-a1de-cedbba7...,989,220,1292,294,CheckBox
4,./model/images/2469da39-65f0-428b-a1de-cedbba7...,804,107,952,168,RadioButton


In [17]:
train_df, test_df = train_test_split(df, test_size = 0.2, random_state = RANDOM_SEED)

In [18]:
ANNOTATIONS_FILE = "annotations.csv"
CLASSES_FILE = "classes.csv"

In [19]:
train_df.to_csv(ANNOTATIONS_FILE, index = False, header = None)

In [20]:
classes = set(['Label', 'Button', 'TextBox', 'ComboBox', 'Link', 'CheckBox', 'Heading', 'Image', 'Paragraph', 'RadioButton'])

with open(CLASSES_FILE, 'w') as f:
  for i, line in enumerate(sorted(classes)):
    f.write('{}, {}\n'.format(line, i))

In [21]:
PRETRAINED_MODEL = './snapshots/_pretrained_model.h5'

URL_MODEL = 'https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5'
urllib.request.urlretrieve(URL_MODEL, PRETRAINED_MODEL)

print('Downloaded pretrained model to ' + PRETRAINED_MODEL)

Downloaded pretrained model to ./snapshots/_pretrained_model.h5


In [22]:
!head annotations.csv

./model/images/f66ced22-8f4e-4c35-8277-e49fba7bf192.png,25,216,289,300,CheckBox
./model/images/52840896-0073-4c1f-9aca-bbd87c17eac9.png,588,441,766,521,Paragraph
./model/images/85c935d2-a69a-4d58-907b-22cf24a53fd5.png,181,989,633,1095,Label
./model/images/d20b2bf2-ee77-404f-8de3-9238284d1957.png,740,253,862,346,Link
./model/images/1803be9d-936c-4429-9301-2a0aaaffbb46.png,364,203,493,257,Label
./model/images/bde0646d-e0d6-497f-b832-cb2bbf907eb1.png,390,347,644,437,TextBox
./model/images/3892666c-c7c7-4b69-9000-9f050d2ca7fb.png,440,170,637,231,ComboBox
./model/images/80ce0ae6-9f89-429b-b2da-ae1b25e45acb.png,67,411,210,474,Paragraph
./model/images/da54575a-4e89-49aa-af1a-c4d8490ba57b.png,597,359,670,448,Button
./model/images/770c6fbf-1046-4de3-a892-c1d53a9209ae.png,29,100,138,154,Label


In [23]:
!python keras-retinanet/keras_retinanet/bin/train.py --freeze-backbone --random-transform --weights {PRETRAINED_MODEL} --batch-size 8 --steps 500 --epochs 10 csv annotations.csv classes.csv

Using TensorFlow backend.
2020-04-07 16:14:15.071686: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libnvinfer.so.6
2020-04-07 16:14:15.072930: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libnvinfer_plugin.so.6
Creating model, this may take a second...
2020-04-07 16:14:15.860524: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-04-07 16:14:17.158240: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1555] Found device 0 with properties: 
pciBusID: 0000:08:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.65GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2020-04-07 16:14:17.158336: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32, numpy=
array([[-22.627417, -11.313708,  22.627417,  11.313708],
       [-28.50876 , -14.25438 ,  28.50876 ,  14.25438 ],
       [-35.918785, -17.959393,  35.918785,  17.959393],
       [-16.      , -16.      ,  16.      ,  16.      ],
       [-20.158737, -20.158737,  20.158737,  20.158737],
       [-25.398417, -25.398417,  25.398417,  25.398417],
       [-11.313708, -22.627417,  11.313708,  22.627417],
       [-14.25438 , -28.50876 ,  14.25438 ,  28.50876 ],
       [-17.959393, -35.918785,  17.959393,  35.918785]], dtype=float32)> anchors
tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32, numpy=
array([[-45.254833, -22.627417,  45.254833,  22.627417],
       [-57.01752 , -28.50876 ,  57.01752 ,  28.50876 ],
       [-71.83757 , -35.918785,  71.83757 ,  35.918785],
       [-32.      , -32.      ,  32.      ,  32.      ],
       [-40.317474, -40.317474,  40.317474,  40.317474],
       [-50.796833, -50.796833,  50.7

Epoch 1/10
2020-04-07 16:14:26.588834: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2020-04-07 16:14:28.265952: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
500/500 [==============================] - 640s 1s/step - loss: 2.1697 - regression_loss: 1.4363 - classification_loss: 0.7335

Epoch 00001: saving model to ./snapshots/resnet50_csv_01.h5
Epoch 2/10
500/500 [==============================] - 621s 1s/step - loss: 1.5761 - regression_loss: 1.0980 - classification_loss: 0.4781

Epoch 00002: saving model to ./snapshots/resnet50_csv_02.h5
Epoch 3/10
500/500 [==============================] - 618s 1s/step - loss: 1.4023 - regression_loss: 0.9990 - classification_loss: 0.4033

Epoch 00003: saving model to ./snapshots/resnet50_csv_03.h5
Epoch 4/10
500/500 [==============================] - 613s 1s/step - loss: 1.2986 - regression_loss: 0.9388 - classif

In [24]:
!ls snapshots

_pretrained_model.h5  resnet50_csv_04.h5  resnet50_csv_08.h5
resnet50_csv_01.h5    resnet50_csv_05.h5  resnet50_csv_09.h5
resnet50_csv_02.h5    resnet50_csv_06.h5  resnet50_csv_10.h5
resnet50_csv_03.h5    resnet50_csv_07.h5


In [25]:
model_path = "snapshots/resnet50_csv_10.h5"

In [26]:
model = models.load_model(model_path, backbone_name="resnet50")
model = models.convert_model(model)

Using TensorFlow backend.


tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32, numpy=
array([[-22.627417, -11.313708,  22.627417,  11.313708],
       [-28.50876 , -14.25438 ,  28.50876 ,  14.25438 ],
       [-35.918785, -17.959393,  35.918785,  17.959393],
       [-16.      , -16.      ,  16.      ,  16.      ],
       [-20.158737, -20.158737,  20.158737,  20.158737],
       [-25.398417, -25.398417,  25.398417,  25.398417],
       [-11.313708, -22.627417,  11.313708,  22.627417],
       [-14.25438 , -28.50876 ,  14.25438 ,  28.50876 ],
       [-17.959393, -35.918785,  17.959393,  35.918785]], dtype=float32)> anchors
tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32, numpy=
array([[-45.254833, -22.627417,  45.254833,  22.627417],
       [-57.01752 , -28.50876 ,  57.01752 ,  28.50876 ],
       [-71.83757 , -35.918785,  71.83757 ,  35.918785],
       [-32.      , -32.      ,  32.      ,  32.      ],
       [-40.317474, -40.317474,  40.317474,  40.317474],
       [-50.796833, -50.796833,  50.7

In [27]:
labels_to_name = pd.read_csv(CLASSES_FILE, header = None).T.loc[0].to_dict()
labels_to_name

{0: 'Button',
 1: 'CheckBox',
 2: 'ComboBox',
 3: 'Heading',
 4: 'Image',
 5: 'Label',
 6: 'Link',
 7: 'Paragraph',
 8: 'RadioButton',
 9: 'TextBox'}

In [73]:


def draw_detections(image, boxes, scores, labels):
  for box, score, label in zip(boxes[0], scores[0], labels[0]):
    if score < THRES_SCORE:
        continue

    color = label_color(label)

    b = box.astype(int)
    draw_box(image, b, color=color)

    caption = "{} {:.3f}".format(labels_to_names[label], score)
    draw_caption(image, b, caption)

In [74]:
THRES_SCORE = 0.4

def show_detected_objects(image_row):
    img_path = image_row.image_name
    true_box = [
        image_row.x_min, image_row.y_min, image_row.x_max, image_row.y_max
    ]
    image = read_image_bgr(img_path)

    draw = image.copy()
    draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

    image = preprocess_image(image)
    image, scale = resize_image(image)

    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis = 0))
    boxes /= scale

    # draw_box(draw, true_box, color=(255, 0, 0))
    draw_detections(image, boxes, scores, labels)
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        if score < THRES_SCORE:
            continue;

        color = label_color(label)

        b = box.astype(int)
        draw_box(draw, b, color = color)

        caption = "{} {:.3f}".format(labels_to_name[label], score)
        draw_caption(draw, b, caption)

    plt.axis('off')
    plt.imshow(draw)
    plt.show()

In [75]:
test_df.head(n=10)

,image_name,x_min,y_min,x_max,y_max,class_name
1394,./model/images/4ffbe90a-7888-4dd3-9e5c-7565fa1...,28,229,544,336,TextBox
353,./model/images/125ccd38-5f68-48c0-b060-8fb08dc...,84,318,348,385,Label
1334,./model/images/ae3de36d-ba93-4a5d-829c-38fa0b8...,338,177,906,288,Label
906,./model/images/5bc04e44-1903-40b3-a782-79035dc...,433,452,784,561,Paragraph
1290,./model/images/12ca11b1-9d0b-475b-96c4-ce30aeb...,107,712,254,784,Label
1274,./model/images/12ca11b1-9d0b-475b-96c4-ce30aeb...,676,111,767,157,Label
939,./model/images/41bd3b56-732a-4a13-a23a-1619dd3...,214,766,733,875,Button
1732,./model/images/05495118-dcee-4e4e-bee1-b682309...,134,540,548,600,ComboBox
65,./model/images/3cf4d5e6-cede-4272-aec0-f29790a...,374,146,662,257,CheckBox
1047,./model/images/6450fc7c-cd20-4f5d-a41d-c23bba8...,691,317,877,392,Label


In [76]:
for i in range(10, 20):
    show_detected_objects(test_df.iloc[i])

NameError: name 'labels_to_names' is not defined